In [1]:
# import stuffs
import pandas as pd
import requests 
import json

In [2]:
# name stuffs
file_one = 'Datafiniti_Fast_Food_Restaurants.csv'
file_two = 'McDonalds.csv'
file_tree = 'FastFoodRestaurants.csv'

# open stuffs
with open(file_one) as dat_data:
    read_dat = pd.read_csv(dat_data)
    read_dat = pd.DataFrame(read_dat)

with open(file_two) as data_data:
    anotha_one = pd.read_csv(data_data)
    anotha_one = pd.DataFrame(anotha_one)
    anotha_two = anotha_one

with open(file_tree) as still_more:
    read_time = pd.read_csv(still_more)
    read_time = pd.DataFrame(read_time)

# read stuffs
read_time = read_time.set_index('name')
read_dat = read_dat.set_index('name')
anotha_one = anotha_one.rename(columns={'properties.addressLine1':'address'})
anotha_one = anotha_one.rename(columns={'properties.subDivision':'state', 'properties.postcode':'zipcode'})

In [3]:
# munge stuffs
mcy_d = read_dat.loc['McDonalds']
mcy_dd = read_time.loc['McDonalds']
mc_d = mcy_d.merge(mcy_dd, how='outer')
mc_d = mc_d.drop_duplicates('address')
mc_d = mc_d.rename(columns={'province':'state'})
mc_d = mc_d.set_index('state')
mc_d = mc_d.rename(columns={'postalCode':'zipcode'})
burger_time = mc_d.merge(anotha_one, how='right', on=['address', 'zipcode'])
daburgas = burger_time[['state', 'address', 'zipcode']]
daburgas = daburgas.drop_duplicates('address', keep='first', inplace=False).set_index('state')

In [4]:
# set up API link
fcc_api = 'https://geo.fcc.gov/api/census/block/find'

# function to look up county information
def where_ru(lat, long):
    params = {
        'latitude': lat,
        'longitude': long,
        'censusYear': 2010,
        'format': 'json'
    }
    cnty_info = requests.get(fcc_api, params=params)
    cnty_info = cnty_info.json()
    return cnty_info
    
# function used to retreive county name
def there_ur(fcc_info):
    over_where = fcc_info['County'].values()
    o_there = list(over_where)[0]
    return o_there

# used to cycle through data and then look up information
def fips_rreal(df):
    alwys_wndrd = []
    df = df.reset_index()
    for y in df.index:
        lats = df.loc[y, 'latitude']
        longs = df.loc[y, 'longitude']
        looky_here = where_ru(lat=lats, long=longs)
        dip_fip = there_ur(fcc_info=looky_here)
        alwys_wndrd.append(dip_fip)
    df['FIPS'] = alwys_wndrd
    return df

poo = fips_rreal(mc_d)


In [5]:
anotha_two['latitude'] = [x.strip(" ' [ ] ' ").split(',')[1] for x in anotha_two['geometry.coordinates'].values]
anotha_two['longitude'] = [x.strip(" ' [ ] ' ").split(',')[0] for x in anotha_two['geometry.coordinates'].values]
 

In [6]:

rfips_real = fips_rreal(anotha_two)

KeyboardInterrupt: 

,properties.id,properties.identifierValue,properties.identifiers.gblnumber,properties.identifiers.storeIdentifier,properties.openstatus,properties.postcode
0,195500307206-en-us,6781,1.955000e+11,"[{'identifierType': 'SiteIdNumber', 'identifie...",OPEN,37203-3117
1,195500306629-en-us,3803,1.955000e+11,"[{'identifierType': 'SiteIdNumber', 'identifie...",OPEN,37203-1426
2,195500306278-en-us,1421,1.955000e+11,"[{'identifierType': 'SiteIdNumber', 'identifie...",OPEN,37204-3008
3,195500306179-en-us,1423,1.955000e+11,"[{'identifierType': 'SiteIdNumber', 'identifie...",OPEN,37210-3510
4,195500308838-en-us,14646,1.955000e+11,"[{'identifierType': 'SiteIdNumber', 'identifie...",OPEN,37228
